In [1]:
import numpy as np  
import navpy
import math
import pandas as pd 
import os, csv, sys
import glob
import subprocess
from tqdm.notebook import tqdm
from tqdm.auto import tqdm
from dataclasses import dataclass
from scipy.interpolate import InterpolatedUnivariateSpline
from datetime import datetime, timezone, timedelta
from decimal import Decimal, getcontext, ROUND_DOWN

folder_path = os.path.abspath('gnss_analysis')
sys.path.append(folder_path)
from gnssutils import EphemerisManager,WLS,Troposphere,Ionosphere,ElevationAzimuthAngle
pd.options.mode.chained_assignment = None  # default='warn'


In [2]:
path = 'gnss_2024_07_05_12_11.csv'
gnss = pd.read_csv(path, index_col=False)

In [3]:
gnss['utcTimeMillis'] = pd.to_numeric(gnss['utcTimeMillis'])
# gnss['utcTimeMillis'] = gnss['utcTimeMillis'].apply(lambda x: round(x, -3))

WeekNanos = 604800000000000
LightSpeed = Decimal('0.299792458')

gnss['BiasNanos'] = pd.to_numeric(gnss['BiasNanos']).astype(float)
gnss['TimeNanos'] = gnss['TimeNanos'].apply(Decimal)
gnss['FullBiasNanos'] = gnss['FullBiasNanos'].apply(Decimal)
gnss['BiasNanos'] = gnss['BiasNanos'].apply(lambda x: Decimal(x).quantize(Decimal('0.001'), rounding=ROUND_DOWN))

# Calculate ArrivalTimeNanosSinceGpsEpoch
gnss['ArrivalTimeNanosSinceGpsEpoch'] = (gnss['TimeNanos'] - gnss['FullBiasNanos'] - gnss['BiasNanos'])
res = gnss['ArrivalTimeNanosSinceGpsEpoch'].iloc[0] % 1000000

gnss['ArrivalTimeNanosSinceGpsEpoch'] = gnss['ArrivalTimeNanosSinceGpsEpoch'].apply(lambda x: round(x, -6)) + res

# Calculate GpsWeekNum
gnss['GpsWeekNum'] = np.floor(gnss['ArrivalTimeNanosSinceGpsEpoch'] / WeekNanos).astype(int)

# Calculate ReceivedSvTimeNanosSinceGpsEpoch
gnss['ReceivedSvTimeNanosSinceGpsEpoch'] = gnss['GpsWeekNum'] * WeekNanos + gnss['ReceivedSvTimeNanos']

gnss['ArrivalTimeNanosSinceGpsEpoch'] = gnss['ArrivalTimeNanosSinceGpsEpoch'].apply(Decimal)
gnss['ReceivedSvTimeNanosSinceGpsEpoch'] = gnss['ReceivedSvTimeNanosSinceGpsEpoch'].apply(Decimal)
gnss['ReceivedSvTimeUncertaintyNanos'] = gnss['ReceivedSvTimeUncertaintyNanos'].apply(Decimal)

gnss['RawPseudorangeMeters'] = (gnss['ArrivalTimeNanosSinceGpsEpoch'] - gnss['ReceivedSvTimeNanosSinceGpsEpoch']) * LightSpeed
gnss['RawPseudorangeUncertaintyMeters'] = gnss['ReceivedSvTimeUncertaintyNanos'] * LightSpeed

def satellite_selection(df):
    idx =  df['RawPseudorangeMeters'] > 0
    idx =  df['RawPseudorangeMeters'] < 32085273
    idx &= df['Cn0DbHz'] > 18.0  # C/N0 (dB-Hz)
    idx &= df['MultipathIndicator'] == 0 # Multipath flag
    idx &= df['ReceivedSvTimeUncertaintyNanos'] < 500
    idx &= df['ConstellationType'] == 1
    return df[idx]
    
utcTimeMillis = gnss['utcTimeMillis'].unique()
nepoch = len(utcTimeMillis)
measurement = pd.DataFrame()
utcTimeLost = []

for i, (t_utc, df) in enumerate(tqdm(gnss.groupby('utcTimeMillis'), total=nepoch)):
    df_pr = satellite_selection(df)
    df_pr['Constellation'] = 'G'
    df_pr['Svid'] = df_pr['Svid'].astype(str)
    df_pr.loc[df_pr['Svid'].str.len() == 1, 'Svid'] = '0' + df_pr['Svid']
    df_pr['SvName'] = df_pr['Constellation'] + df_pr['Svid']
    df_pr = df_pr.drop_duplicates(subset='SvName')

    if len(df_pr) < 4:
        continue
    measurement = pd.concat([measurement, df_pr])

  0%|          | 0/140 [00:00<?, ?it/s]

In [4]:
measurement = measurement.reset_index(drop=True)
# pd.set_option('display.float_format', lambda x: f'{x:.0f}' if x == int(x) else f'{x}')
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

measurement

,utcTimeMillis,TimeNanos,LeapSecond,TimeUncertaintyNanos,FullBiasNanos,BiasNanos,BiasUncertaintyNanos,DriftNanosPerSecond,DriftUncertaintyNanosPerSecond,HardwareClockDiscontinuityCount,...,CodeType,ChipsetElapsedRealtimeNanos,SignalType,ArrivalTimeNanosSinceGpsEpoch,ReceivedSvTimeNanosSinceGpsEpoch,RawPseudorangeMeters,RawPseudorangeUncertaintyMeters,GpsWeekNum,Constellation,SvName
0,1720156318000,624412883000000,18,0.0,-1403567123529528456,0.176,38.762078,10.021986,14.815086,272.0,...,C,900459050683616,GPS,1404191536413528455.824,1404191536343111680,21110418.308711935392,10.492736030,2321,G,G05
1,1720156318000,624412883000000,18,0.0,-1403567123529528456,0.176,38.762078,10.021986,14.815086,272.0,...,C,900459050683616,GPS,1404191536413528455.824,1404191536339133440,22303064.656825855392,4.197094412,2321,G,G11
2,1720156318000,624412883000000,18,0.0,-1403567123529528456,0.176,38.762078,10.021986,14.815086,272.0,...,C,900459050683616,GPS,1404191536413528455.824,1404191536335416832,23417275.704568319392,11.991698320,2321,G,G12
3,1720156318000,624412883000000,18,0.0,-1403567123529528456,0.176,38.762078,10.021986,14.815086,272.0,...,C,900459050683616,GPS,1404191536413528455.824,1404191536342466048,21303973.912955391392,6.295641618,2321,G,G13
4,1720156318000,624412883000000,18,0.0,-1403567123529528456,0.176,38.762078,10.021986,14.815086,272.0,...,C,900459050683616,GPS,1404191536413528455.824,1404191536338121984,22606291.537224703392,5.096471786,2321,G,G15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,1720156457000,624551883000000,18,0.0,-1403567123529528991,0.850,39.878189,-9.259875,18.287176,272.0,...,C,900598050684151,GPS,1404191675413528455.824,1404191675342665728,21244111.354941951392,3.897301954,2321,G,G13
1098,1720156457000,624551883000000,18,0.0,-1403567123529528991,0.850,39.878189,-9.259875,18.287176,272.0,...,C,900598050684151,GPS,1404191675413528455.824,1404191675338419968,22516958.181420031392,11.991698320,2321,G,G15
1099,1720156457000,624551883000000,18,0.0,-1403567123529528991,0.850,39.878189,-9.259875,18.287176,272.0,...,C,900598050684151,GPS,1404191675413528455.824,1404191675331771648,24510074.375790591392,5.696056702,2321,G,G19
1100,1720156457000,624551883000000,18,0.0,-1403567123529528991,0.850,39.878189,-9.259875,18.287176,272.0,...,C,900598050684151,GPS,1404191675413528455.824,1404191675340890368,21776350.893176831392,7.195018992,2321,G,G20


In [5]:
filtered_measurement = measurement.dropna(subset=['ArrivalTimeNanosSinceGpsEpoch'])
filtered_measurement = filtered_measurement.sort_values(by=['utcTimeMillis','Svid']).reset_index(drop=True)
filtered_measurement

,utcTimeMillis,TimeNanos,LeapSecond,TimeUncertaintyNanos,FullBiasNanos,BiasNanos,BiasUncertaintyNanos,DriftNanosPerSecond,DriftUncertaintyNanosPerSecond,HardwareClockDiscontinuityCount,...,CodeType,ChipsetElapsedRealtimeNanos,SignalType,ArrivalTimeNanosSinceGpsEpoch,ReceivedSvTimeNanosSinceGpsEpoch,RawPseudorangeMeters,RawPseudorangeUncertaintyMeters,GpsWeekNum,Constellation,SvName
0,1720156318000,624412883000000,18,0.0,-1403567123529528456,0.176,38.762078,10.021986,14.815086,272.0,...,C,900459050683616,GPS,1404191536413528455.824,1404191536343111680,21110418.308711935392,10.492736030,2321,G,G05
1,1720156318000,624412883000000,18,0.0,-1403567123529528456,0.176,38.762078,10.021986,14.815086,272.0,...,C,900459050683616,GPS,1404191536413528455.824,1404191536339133440,22303064.656825855392,4.197094412,2321,G,G11
2,1720156318000,624412883000000,18,0.0,-1403567123529528456,0.176,38.762078,10.021986,14.815086,272.0,...,C,900459050683616,GPS,1404191536413528455.824,1404191536335416832,23417275.704568319392,11.991698320,2321,G,G12
3,1720156318000,624412883000000,18,0.0,-1403567123529528456,0.176,38.762078,10.021986,14.815086,272.0,...,C,900459050683616,GPS,1404191536413528455.824,1404191536342466048,21303973.912955391392,6.295641618,2321,G,G13
4,1720156318000,624412883000000,18,0.0,-1403567123529528456,0.176,38.762078,10.021986,14.815086,272.0,...,C,900459050683616,GPS,1404191536413528455.824,1404191536338121984,22606291.537224703392,5.096471786,2321,G,G15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,1720156457000,624551883000000,18,0.0,-1403567123529528991,0.850,39.878189,-9.259875,18.287176,272.0,...,C,900598050684151,GPS,1404191675413528455.824,1404191675342665728,21244111.354941951392,3.897301954,2321,G,G13
1098,1720156457000,624551883000000,18,0.0,-1403567123529528991,0.850,39.878189,-9.259875,18.287176,272.0,...,C,900598050684151,GPS,1404191675413528455.824,1404191675338419968,22516958.181420031392,11.991698320,2321,G,G15
1099,1720156457000,624551883000000,18,0.0,-1403567123529528991,0.850,39.878189,-9.259875,18.287176,272.0,...,C,900598050684151,GPS,1404191675413528455.824,1404191675331771648,24510074.375790591392,5.696056702,2321,G,G19
1100,1720156457000,624551883000000,18,0.0,-1403567123529528991,0.850,39.878189,-9.259875,18.287176,272.0,...,C,900598050684151,GPS,1404191675413528455.824,1404191675340890368,21776350.893176831392,7.195018992,2321,G,G20


In [6]:
#lấy dữ liệu ephemeris
ephemeris_data_directory = 'output'
manager = EphemerisManager(ephemeris_data_directory)

# while num_sat < 4:
#     one_epoch = measurement.loc[measurement['Epoch'] == epoch].drop_duplicates(subset='SvName')
    
#     if not one_epoch.empty:
#         timestamp = pd.to_datetime(one_epoch.iloc[0]['UnixTime'], unit='s')
#         one_epoch.set_index('SvName', inplace=True)
#         num_sat = len(one_epoch.index)
    
#     epoch += 1

filtered_measurement['UnixTime'] = pd.to_datetime(filtered_measurement['utcTimeMillis'],unit='ms', utc=True)
timestamps = pd.to_datetime(filtered_measurement['UnixTime'].unique(),unit='s')
ephemeris_data = pd.DataFrame()

for timestamp in timestamps:
    # Get satellite names for the current timestamp
    sats = filtered_measurement.loc[filtered_measurement['UnixTime'] == timestamp, 'SvName'].tolist()
    # Retrieve ephemeris data
    ephemeris = manager.get_ephemeris(timestamp, sats)
    ephemeris_data = pd.concat([ephemeris_data, ephemeris])
#ephemeris = manager.get_ephemeris(timestamp, sats)
#timestamp
ephemeris_data.reset_index(drop=True, inplace=True)
ephemeris_data

Retrieving gnss/data/daily/2024/brdc/brdc1870.24n.gz from https://cddis.nasa.gov/archive/
File downloaded: output\nasa\brdc1870.24n.gz


,time,SVclockBias,SVclockDrift,SVclockDriftRate,IODE,C_rs,deltaN,M_0,C_uc,e,...,L2Pflag,SVacc,health,TGD,IODC,TransTime,FitIntvl,source,t_oc,Leap Seconds
0,2024-07-05 04:00:00+00:00,-0.000179,-1.364242e-12,0.0,11.0,-148.75000,4.060883e-09,-1.044364,-7.402152e-06,0.005948,...,0.0,2.0,0.0,-1.071021e-08,11.0,439218.0,4.0,output\nasa\brdc1870.24n,446400.0,18
1,2024-07-05 04:00:00+00:00,-0.000699,-7.275958e-12,0.0,0.0,26.71875,4.416255e-09,-2.097397,1.488253e-06,0.001426,...,0.0,2.0,0.0,-8.847564e-09,768.0,439218.0,4.0,output\nasa\brdc1870.24n,446400.0,18
2,2024-07-05 04:00:00+00:00,-0.000520,-3.183231e-12,0.0,49.0,16.50000,4.482687e-09,1.302942,5.867332e-07,0.008694,...,0.0,2.0,0.0,-1.257285e-08,49.0,440418.0,4.0,output\nasa\brdc1870.24n,446400.0,18
3,2024-07-05 04:00:00+00:00,0.000662,2.501110e-12,0.0,29.0,-37.09375,4.187674e-09,-1.672752,-1.924112e-06,0.008258,...,0.0,2.0,0.0,-1.117587e-08,29.0,439218.0,4.0,output\nasa\brdc1870.24n,446400.0,18
4,2024-07-05 04:00:00+00:00,0.000178,4.206413e-12,0.0,78.0,-79.15625,4.700910e-09,-2.356005,-4.215166e-06,0.015805,...,0.0,2.0,0.0,-1.024455e-08,78.0,439218.0,4.0,output\nasa\brdc1870.24n,446400.0,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,2024-07-05 04:00:00+00:00,0.000662,2.501110e-12,0.0,29.0,-37.09375,4.187674e-09,-1.672752,-1.924112e-06,0.008258,...,0.0,2.0,0.0,-1.117587e-08,29.0,439218.0,4.0,output\nasa\brdc1870.24n,446400.0,18
1098,2024-07-05 04:00:00+00:00,0.000178,4.206413e-12,0.0,78.0,-79.15625,4.700910e-09,-2.356005,-4.215166e-06,0.015805,...,0.0,2.0,0.0,-1.024455e-08,78.0,439218.0,4.0,output\nasa\brdc1870.24n,446400.0,18
1099,2024-07-05 04:00:00+00:00,0.000506,5.456968e-12,0.0,161.0,143.71875,4.114457e-09,0.547825,7.163733e-06,0.009978,...,0.0,2.0,0.0,-1.536682e-08,161.0,440748.0,4.0,output\nasa\brdc1870.24n,446400.0,18
1100,2024-07-05 04:00:00+00:00,0.000373,-4.547474e-13,0.0,31.0,-150.09375,4.475544e-09,-2.972955,-7.648021e-06,0.003678,...,0.0,2.0,0.0,-8.381903e-09,31.0,439218.0,4.0,output\nasa\brdc1870.24n,446400.0,18


In [7]:
LIGHTSPEED = 2.99792458e8

def calculate_satellite_position(ephemeris, transmit_time):
    mu = 3.986005e14
    OmegaDot_e = 7.2921151467e-5
    F = -4.442807633e-10
    sv_position = pd.DataFrame()
    sv_position['sv']= ephemeris.index
    sv_position.set_index('sv', inplace=True)
    sv_position['t_k'] = transmit_time - ephemeris['t_oe']
    A = ephemeris['sqrtA'].pow(2)
    n_0 = np.sqrt(mu / A.pow(3))
    n = n_0 + ephemeris['deltaN']
    M_k = ephemeris['M_0'] + n * sv_position['t_k']
    E_k = M_k
    err = pd.Series(data=[1]*len(sv_position.index))
    i = 0
    while err.abs().min() > 1e-8 and i < 10:
        new_vals = M_k + ephemeris['e']*np.sin(E_k)
        err = new_vals - E_k
        E_k = new_vals
        i += 1
        
    sinE_k = np.sin(E_k)
    cosE_k = np.cos(E_k)
    delT_r = F * ephemeris['e'].pow(ephemeris['sqrtA']) * sinE_k
    delT_oc = transmit_time - ephemeris['t_oc']
    sv_position['delT_sv'] = ephemeris['SVclockBias'] + ephemeris['SVclockDrift'] * delT_oc + ephemeris['SVclockDriftRate'] * delT_oc.pow(2)

    v_k = np.arctan2(np.sqrt(1-ephemeris['e'].pow(2))*sinE_k,(cosE_k - ephemeris['e']))

    Phi_k = v_k + ephemeris['omega']

    sin2Phi_k = np.sin(2*Phi_k)
    cos2Phi_k = np.cos(2*Phi_k)

    du_k = ephemeris['C_us']*sin2Phi_k + ephemeris['C_uc']*cos2Phi_k
    dr_k = ephemeris['C_rs']*sin2Phi_k + ephemeris['C_rc']*cos2Phi_k
    di_k = ephemeris['C_is']*sin2Phi_k + ephemeris['C_ic']*cos2Phi_k

    u_k = Phi_k + du_k

    r_k = A*(1 - ephemeris['e']*np.cos(E_k)) + dr_k

    i_k = ephemeris['i_0'] + di_k + ephemeris['IDOT']*sv_position['t_k']

    x_k_prime = r_k*np.cos(u_k)
    y_k_prime = r_k*np.sin(u_k)

    Omega_k = ephemeris['Omega_0'] + (ephemeris['OmegaDot'] - OmegaDot_e)*sv_position['t_k'] - OmegaDot_e*ephemeris['t_oe']

    sv_position['x_k'] = x_k_prime*np.cos(Omega_k) - y_k_prime*np.cos(i_k)*np.sin(Omega_k)
    sv_position['y_k'] = x_k_prime*np.sin(Omega_k) + y_k_prime*np.cos(i_k)*np.cos(Omega_k)
    sv_position['z_k'] = y_k_prime*np.sin(i_k)
    sv_position['SvClockBiasMeters'] = ephemeris['SVclockBias'] * LIGHTSPEED
    sv_position['SvClockDriftMetersPerSecond'] = ephemeris['SVclockDrift'] * LIGHTSPEED

    #calculate velocity
    Edot_k = n_0/(1-ephemeris['e']*np.cos(E_k))
    
    vdot_k = Edot_k * np.sqrt(1 - ephemeris['e']**2)/(1 - ephemeris['e']*np.cos(E_k))
    
    d_idotk = ephemeris['IDOT'] + 2 * vdot_k * (ephemeris['C_is'] * cos2Phi_k - ephemeris['C_ic']*sin2Phi_k)
    
    udot_k = vdot_k + 2 * vdot_k * (ephemeris['C_us']*cos2Phi_k - ephemeris['C_uc']*sin2Phi_k)
    
    rdot_k = ephemeris['e'] * A * Edot_k * np.sin(E_k) + 2 * vdot_k * (ephemeris['C_rs'] * cos2Phi_k - ephemeris['C_rc'] * sin2Phi_k)
    
    Omegadot_k = ephemeris['OmegaDot'] - OmegaDot_e
    
    x_plus_dot_k = rdot_k * np.cos(u_k) - r_k * udot_k*np.sin(u_k)
    y_plus_dot_k = rdot_k * np.sin(u_k) + r_k * udot_k*np.cos(u_k)
    
    sv_position['v_x'] = -x_k_prime * Omegadot_k * np.sin(Omega_k) + x_plus_dot_k * np.cos(Omega_k) - y_plus_dot_k * np.sin(Omega_k)*np.cos(i_k) - y_k_prime * (Omegadot_k * np.cos(Omega_k) * np.cos(i_k) - d_idotk * np.sin(Omega_k)*np.sin(i_k))
    
    sv_position['v_y'] = x_k_prime * Omegadot_k * np.cos(Omega_k) + x_plus_dot_k * np.sin(Omega_k) + y_plus_dot_k * np.cos(Omega_k)*np.cos(i_k) - y_k_prime * (Omegadot_k * np.sin(Omega_k) * np.cos(i_k) + d_idotk * np.cos(Omega_k)*np.sin(i_k))
    
    sv_position['v_z'] = y_plus_dot_k * np.sin(i_k) + y_k_prime * d_idotk * np.cos(i_k)

    return sv_position


In [8]:
# Run the function and check out the results:
filtered_measurement['ReceivedSvTimeNanos'] = pd.to_numeric(filtered_measurement['ReceivedSvTimeNanos'])
filtered_measurement['TimeOffsetNanos'] = pd.to_numeric(filtered_measurement['TimeOffsetNanos'])
sv_position = calculate_satellite_position(ephemeris_data, (filtered_measurement['ReceivedSvTimeNanos'] + filtered_measurement['TimeOffsetNanos'])*1e-9)
filtered_measurement['SvPositionXEcefMeters'] = sv_position['x_k']
filtered_measurement['SvPositionYEcefMeters'] = sv_position['y_k']
filtered_measurement['SvPositionZEcefMeters'] = sv_position['z_k']
filtered_measurement['SvVelocityXEcefMetersPerSecond'] = sv_position['v_x']
filtered_measurement['SvVelocityYEcefMetersPerSecond'] = sv_position['v_y']
filtered_measurement['SvVelocityZEcefMetersPerSecond'] = sv_position['v_z']
filtered_measurement['SvClockBiasMeters'] = sv_position['SvClockBiasMeters']
filtered_measurement['SvClockDriftMetersPerSecond'] = sv_position['SvClockDriftMetersPerSecond']
filtered_measurement

,utcTimeMillis,TimeNanos,LeapSecond,TimeUncertaintyNanos,FullBiasNanos,BiasNanos,BiasUncertaintyNanos,DriftNanosPerSecond,DriftUncertaintyNanosPerSecond,HardwareClockDiscontinuityCount,...,SvName,UnixTime,SvPositionXEcefMeters,SvPositionYEcefMeters,SvPositionZEcefMeters,SvVelocityXEcefMetersPerSecond,SvVelocityYEcefMetersPerSecond,SvVelocityZEcefMetersPerSecond,SvClockBiasMeters,SvClockDriftMetersPerSecond
0,1720156318000,624412883000000,18,0.0,-1403567123529528456,0.176,38.762078,10.021986,14.815086,272.0,...,G05,2024-07-05 05:11:58+00:00,1.004110e+06,2.055465e+07,1.656088e+07,-1398.197626,-1626.925907,2089.044672,-53559.045897,-0.000409
1,1720156318000,624412883000000,18,0.0,-1403567123529528456,0.176,38.762078,10.021986,14.815086,272.0,...,G11,2024-07-05 05:11:58+00:00,-1.589198e+07,1.222614e+07,1.741362e+07,-2209.064970,-136.868346,-1928.146781,-209528.952453,-0.002181
2,1720156318000,624412883000000,18,0.0,-1403567123529528456,0.176,38.762078,10.021986,14.815086,272.0,...,G12,2024-07-05 05:11:58+00:00,9.125840e+06,2.436024e+07,-5.762591e+06,-567.980927,-477.602684,-3062.898189,-155794.008361,-0.000954
3,1720156318000,624412883000000,18,0.0,-1403567123529528456,0.176,38.762078,10.021986,14.815086,272.0,...,G13,2024-07-05 05:11:58+00:00,-1.344144e+07,2.262061e+07,-2.697683e+06,-398.113515,111.493681,3191.292594,198598.694468,0.000750
4,1720156318000,624412883000000,18,0.0,-1403567123529528456,0.176,38.762078,10.021986,14.815086,272.0,...,G15,2024-07-05 05:11:58+00:00,-1.107445e+06,2.504804e+07,-8.976314e+06,-645.907410,927.253531,2845.697546,53230.004591,0.001261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,1720156457000,624551883000000,18,0.0,-1403567123529528991,0.850,39.878189,-9.259875,18.287176,272.0,...,G13,2024-07-05 05:14:17+00:00,-1.349454e+07,2.263311e+07,-2.253550e+06,-365.911454,68.198985,3198.708685,198598.694468,0.000750
1098,1720156457000,624551883000000,18,0.0,-1403567123529528991,0.850,39.878189,-9.259875,18.287176,272.0,...,G15,2024-07-05 05:14:17+00:00,-1.195777e+06,2.517400e+07,-8.578944e+06,-625.007464,884.994683,2871.465960,53230.004591,0.001261
1099,1720156457000,624551883000000,18,0.0,-1403567123529528991,0.850,39.878189,-9.259875,18.287176,272.0,...,G19,2024-07-05 05:14:17+00:00,-1.889214e+07,1.282695e+07,-1.338221e+07,700.025404,-1550.529277,-2546.100625,151672.546134,0.001636
1100,1720156457000,624551883000000,18,0.0,-1403567123529528991,0.850,39.878189,-9.259875,18.287176,272.0,...,G20,2024-07-05 05:14:17+00:00,-7.225350e+06,1.423780e+07,2.130727e+07,-1909.497881,-1896.732177,606.941531,111854.220490,-0.000136


In [9]:
columns_to_merge = [
    'utcTimeMillis', 'Svid','SvPositionXEcefMeters', 'SvPositionYEcefMeters', 'SvPositionZEcefMeters',
    'SvVelocityXEcefMetersPerSecond', 'SvVelocityYEcefMetersPerSecond', 'SvVelocityZEcefMetersPerSecond',
    'SvClockBiasMeters', 'SvClockDriftMetersPerSecond']
filtered_subset = filtered_measurement[columns_to_merge]

# filtered_measurement.set_index(['utcTimeMillis', 'Svid'], inplace=True)

# measurement.set_index(['utcTimeMillis', 'Svid'], inplace=True)
# missing_data = pd.DataFrame({
#     0: [0.0] * len(indices_with_zero),
#     'index': indices_with_zero
# })
df_lost = pd.DataFrame({'utcTimeMillis': utcTimeLost, 'Svid': 0* len(utcTimeLost)})
df_combined = pd.concat([filtered_subset, df_lost]).sort_values(by=['utcTimeMillis','Svid']).reset_index(drop=True)

merged_measurement = pd.merge(measurement, df_combined, on=['utcTimeMillis', 'Svid'], how='left')

#pd.set_option('display.float_format', lambda x: f'{x:.0f}' if x == int(x) else f'{x}')
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)
merged_measurement

,utcTimeMillis,TimeNanos,LeapSecond,TimeUncertaintyNanos,FullBiasNanos,BiasNanos,BiasUncertaintyNanos,DriftNanosPerSecond,DriftUncertaintyNanosPerSecond,HardwareClockDiscontinuityCount,...,Constellation,SvName,SvPositionXEcefMeters,SvPositionYEcefMeters,SvPositionZEcefMeters,SvVelocityXEcefMetersPerSecond,SvVelocityYEcefMetersPerSecond,SvVelocityZEcefMetersPerSecond,SvClockBiasMeters,SvClockDriftMetersPerSecond
0,1720156318000,624412883000000,18,0.0,-1403567123529528456,0.176,38.762078,10.021986,14.815086,272.0,...,G,G05,1.004110e+06,2.055465e+07,1.656088e+07,-1398.197626,-1626.925907,2089.044672,-53559.045897,-0.000409
1,1720156318000,624412883000000,18,0.0,-1403567123529528456,0.176,38.762078,10.021986,14.815086,272.0,...,G,G11,-1.589198e+07,1.222614e+07,1.741362e+07,-2209.064970,-136.868346,-1928.146781,-209528.952453,-0.002181
2,1720156318000,624412883000000,18,0.0,-1403567123529528456,0.176,38.762078,10.021986,14.815086,272.0,...,G,G12,9.125840e+06,2.436024e+07,-5.762591e+06,-567.980927,-477.602684,-3062.898189,-155794.008361,-0.000954
3,1720156318000,624412883000000,18,0.0,-1403567123529528456,0.176,38.762078,10.021986,14.815086,272.0,...,G,G13,-1.344144e+07,2.262061e+07,-2.697683e+06,-398.113515,111.493681,3191.292594,198598.694468,0.000750
4,1720156318000,624412883000000,18,0.0,-1403567123529528456,0.176,38.762078,10.021986,14.815086,272.0,...,G,G15,-1.107445e+06,2.504804e+07,-8.976314e+06,-645.907410,927.253531,2845.697546,53230.004591,0.001261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,1720156457000,624551883000000,18,0.0,-1403567123529528991,0.850,39.878189,-9.259875,18.287176,272.0,...,G,G13,-1.349454e+07,2.263311e+07,-2.253550e+06,-365.911454,68.198985,3198.708685,198598.694468,0.000750
1098,1720156457000,624551883000000,18,0.0,-1403567123529528991,0.850,39.878189,-9.259875,18.287176,272.0,...,G,G15,-1.195777e+06,2.517400e+07,-8.578944e+06,-625.007464,884.994683,2871.465960,53230.004591,0.001261
1099,1720156457000,624551883000000,18,0.0,-1403567123529528991,0.850,39.878189,-9.259875,18.287176,272.0,...,G,G19,-1.889214e+07,1.282695e+07,-1.338221e+07,700.025404,-1550.529277,-2546.100625,151672.546134,0.001636
1100,1720156457000,624551883000000,18,0.0,-1403567123529528991,0.850,39.878189,-9.259875,18.287176,272.0,...,G,G20,-7.225350e+06,1.423780e+07,2.130727e+07,-1909.497881,-1896.732177,606.941531,111854.220490,-0.000136


In [10]:
file_name_parts = path.rsplit('.', 1)  # Split the file name to separate the name and the extension
output_file_name = f"{file_name_parts[0]}_sat.{file_name_parts[1]}"  # Add '_sat' before the file extension

# Save the filtered data to the new file
merged_measurement.to_csv(output_file_name, index=False)

In [11]:
# epoch = 0
# num_sat = 0
# while num_sat < 4:
#     one_epoch = measurement.loc[measurement['Epoch'] == epoch]
#     if not one_epoch.empty:
#         one_epoch.set_index('SvName', inplace=True)
#         num_sat = len(one_epoch.index)
#     epoch += 1


In [ ]:
# wls = WLS()
# ecef_list = []
# for epoch in measurement['Epoch'].unique():
#     one_epoch = measurement.loc[(measurement['Epoch'] == epoch)] 
#     if len(one_epoch.index) > 4:
#         x = wls.WLS_onePosition_rawPseudo(one_epoch)
#         ecef_list.append(x)
# ecef_list

In [ ]:
# ecef_array = np.stack(ecef_list, axis=0)
# lla_array = np.stack(navpy.ecef2lla(ecef_array), axis=1)
# lla_array

# # Convert back to Pandas and save to csv
# lla_df = pd.DataFrame(lla_array, columns=['Latitude', 'Longitude', 'Altitude'])
# time_get_file = path.split('_', 1)[1].rsplit('.', 1)[0]  # Get the part after the first underscore and before the extension
# new_output_file_name = f"calculated_position_wls_{time_get_file}.csv"
# lla_df.to_csv(new_output_file_name, index=False)